In [1]:
!pip install catboost
# !pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.3 MB/s eta 0:00:00


In [124]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.ensemble import GradientBoostingRegressor
# import lightgbm as lgb
import math
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
# import optuna
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder

In [125]:
pd.options.display.float_format = '{:.2f}'.format

In [126]:
df = pd.read_csv('./train.csv')
print(df.shape)
print(df.columns)

(54273, 13)
Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price'],
      dtype='object')


In [127]:
# 対数を取ったが下がった。
# df["log_price"] = df["price"].apply(lambda x : math.log(x))

In [128]:
def make_milage_class(x):
    if x <= 100000:
        return "cls_a"
    elif 100000 <= x <= 250000:
        return "cls_b"
    else:
        return 'cls_c'
df["milage_cls"] = df["milage"].apply(make_milage_class)

In [129]:
# df[["milage", 'price']].plot.scatter(x="milage", y='price')

In [130]:
def make_my_class(x):
    if x <= 2000:
        return "cls_a"
    else:
        return 'cls_b'
df["model_year_cls"] = df["model_year"].apply(make_my_class)

In [131]:
# df[["model_year", 'price']].plot.scatter(x="model_year", y='price')

In [132]:
brand_data = pd.DataFrame(df[["price", "brand"]].groupby("brand").agg({'price': ['count', 'median', 'mean', 'std', 'min', 'max']}).reset_index(drop=False))
brand_data.columns = ['_b_'.join(col).strip() for col in brand_data.columns.values]
brand_data = brand_data.fillna(0)
brand_data
df = pd.merge(df, brand_data, how="left", left_on="brand", right_on="brand_b_")

In [133]:
model_data = pd.DataFrame(df[["price", "model"]].groupby("model").agg({'price': ['median', 'mean', 'std']}).reset_index(drop=False))
model_data.columns = ['_m_'.join(col).strip() for col in model_data.columns.values]
model_data = model_data.fillna(0)
model_data
df = pd.merge(df, model_data, how="left", left_on="model", right_on="model_m_")

In [134]:
df["accident_flg"] = df["accident"].apply(lambda x: 0 if x == "None reported" else 1)

In [135]:
def make_transmission(x):
    trans_num_list = [11,10,9,8,7,6,5,4,3,2,1]
    for i in trans_num_list:
        if str(i) in x:
            result = i
            break
        else:
            result = 6
    return result

def make_auto_manual(x):
    if ("A/T" in x) or ("Automatic" in x):
        result = "A/T"
    elif ("M/T" in x) or ("Manual" in x) or ("Mt" in x):
        result = "M/T"
    elif ("Dual Shift Mode" in x) or ("At/Mt" in x):
        result = "DSM"
    elif "Overdrive Switch" in x:
        result = "OS"
    elif "CVT" in x:
        result = "CVT"
    else:
        result = "A/T"
    return result

df["transmission_a"] = df["transmission"].apply(make_transmission)
df["transmission_b"] = df["transmission"].apply(make_auto_manual)

In [136]:
def make_simple_color(x):
    color_list = [
        'Blue', 'Black', 'Purple', 'Gray', 'White', 'Red', 'Silver', 'Green', 'Orange','Brown','Beige', 'Gold',
        "Metallic", "Yellow"
    ]
    for col in color_list:
        if col in x:
            result = col
            break
        else:
            result = "White"
    return result

df["sim_ext_col"] = df["ext_col"].apply(make_simple_color)
df["sim_int_col"] = df["int_col"].apply(make_simple_color)

In [137]:
df["sim_ext_col"].value_counts()

sim_ext_col
Black       15675
White       14044
Gray         8019
Silver       5361
Blue         4793
Red          3271
Green         782
Gold          484
Beige         460
Brown         390
Orange        330
Yellow        306
Metallic      264
Purple         94
Name: count, dtype: int64

In [138]:
df[df["sim_int_col"] == ""]["int_col"].value_counts()

Series([], Name: count, dtype: int64)

In [139]:
df["year_diff"] = 2024 - df["model_year"]
df["year_diff_milage"] = df["year_diff"] * df["milage"]

In [140]:
def make_engine_hp(text, target):
    pattern = rf'(\d+\.?\d*)\s*{target}'

    # 正規表現で検索
    match = re.search(pattern, text)

    # マッチした場合はその数字を返す
    if match:
        return float(match.group(1))
    else:
        return 330 # 平均値

def make_engine_L(text, target):
    pattern = rf'(\d+\.?\d*)\s*{target}'

    # 正規表現で検索
    match = re.search(pattern, text)

    # マッチした場合はその数字を返す
    if match:
        return float(match.group(1))
    else:
        return 3.7 # 平均値

def make_engine_C(text, target):
    pattern = rf'(\d+\.?\d*)\s*{target}'

    # 正規表現で検索
    match = re.search(pattern, text)

    # マッチした場合はその数字を返す
    if match:
        return float(match.group(1))
    else:
        return 6 # 平均値


df["engine_hp"] = df.apply(lambda row: make_engine_hp(row["engine"], "HP"), axis=1)
df["engine_L"] = df.apply(lambda row: make_engine_L(row["engine"], "L"), axis=1)
df["engine_C"] = df.apply(lambda row: make_engine_C(row["engine"], " Cylinder"), axis=1)

In [141]:
for c in ["transmission_a", 'engine_hp', 'engine_L', 'engine_C']:
    brand_data = pd.DataFrame(df[[c , "brand"]].groupby("brand").agg({c : ['median', 'mean', 'std', "min", 'max']}).reset_index(drop=False))
    k = "_" + c + "_"
    brand_k = "brand" + k
    brand_data.columns = [k.join(col).strip() for col in brand_data.columns.values]
    brand_data = brand_data.fillna(0)
    df = pd.merge(df, brand_data, how="left", left_on="brand", right_on=brand_k)

In [142]:
df.columns

Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price', 'milage_cls', 'model_year_cls', 'brand_b_', 'price_b_count',
       'price_b_median', 'price_b_mean', 'price_b_std', 'price_b_min',
       'price_b_max', 'model_m_', 'price_m_median', 'price_m_mean',
       'price_m_std', 'accident_flg', 'transmission_a', 'transmission_b',
       'sim_ext_col', 'sim_int_col', 'year_diff', 'year_diff_milage',
       'engine_hp', 'engine_L', 'engine_C', 'brand_transmission_a_',
       'transmission_a_transmission_a_median',
       'transmission_a_transmission_a_mean',
       'transmission_a_transmission_a_std',
       'transmission_a_transmission_a_min',
       'transmission_a_transmission_a_max', 'brand_engine_hp_',
       'engine_hp_engine_hp_median', 'engine_hp_engine_hp_mean',
       'engine_hp_engine_hp_std', 'engine_hp_engine_hp_min',
       'engine_hp_engine_hp_max', 'brand_engine_L_'

In [143]:
X = df[['brand', 'model_year', 'model', 'milage', 'fuel_type',
        'engine_hp', 'engine_L', 'engine_C',
        'transmission', 'transmission_a', 'transmission_b',
        'ext_col', 'int_col', 'sim_ext_col', 'sim_int_col',
        'accident',
        'year_diff',
        # 'milage_cls', 'model_year_cls',
        'price_b_count','price_b_median', 'price_b_mean', 'price_b_std','price_b_min','price_b_max'
        # 'price_m_median', 'price_m_mean', 'price_m_std',

        # 'clean_title'
        ]]
y = df["price"]
# 訓練セットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

# カテゴリカル特徴量のインデックスを取得
categorical_features_indices = np.where(X_train.dtypes == 'object')[0]

# CatBoostモデルの作成
# model = CatBoostRegressor(silent=True, cat_features=categorical_features_indices)
model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=4, cat_features=categorical_features_indices, verbose=0)

# モデルの訓練
model.fit(X_train, y_train)

# ハイパーパラメータの候補を定義
# param_grid = {
#     'iterations': [100, 200, 300],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'depth': [4, 6, 8],
#     'l2_leaf_reg': [1, 3, 5, 7, 9]
# }

# # GridSearchCVによるハイパーパラメータのチューニング
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
# grid_search.fit(X_train, y_train)

# # 最適なハイパーパラメータ
# best_params = grid_search.best_params_
# print(f'Best parameters: {best_params}')

# # 最適なパラメータでモデルを再作成
# best_model = grid_search.best_estimator_

# # テストセットを使った予測
# y_pred = best_model.predict(X_test)

y_pred = model.predict(X_test)

# RMSEの計算
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse:.2f}')

RMSE: 51544.17


In [78]:
# y_pred = model.predict(X_test)

# # RMSEの計算
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print(f'RMSE: {rmse:.2f}')

In [79]:
# X = df[['brand', 'model_year', 'model', 'milage', 'fuel_type',
#         'engine_hp', 'engine_L', 'engine_C',
#         'transmission', 'transmission_a', 'transmission_b',
#         'ext_col', 'int_col', 'sim_ext_col', 'sim_int_col',
#         'accident',
#         'year_diff',
#         # 'milage_cls', 'model_year_cls',
#         # 'price_b_count',
#         'price_b_median', 'price_b_mean', 'price_b_std','price_b_min', 'price_b_max',
#         # 'price_m_median', 'price_m_mean', 'price_m_std',
#         'year_diff_milage',
#     #    'brand_transmission_a_', 'transmission_a_transmission_a_median',
#     #    'transmission_a_transmission_a_mean',
#     #    'transmission_a_transmission_a_std',
#     #    'transmission_a_transmission_a_min',
#     #    'transmission_a_transmission_a_max', 'brand_engine_hp_',
#     #    'engine_hp_engine_hp_median', 'engine_hp_engine_hp_mean',
#     #    'engine_hp_engine_hp_std', 'engine_hp_engine_hp_min',
#     #    'engine_hp_engine_hp_max', 'brand_engine_L_',
#     #    'engine_L_engine_L_median', 'engine_L_engine_L_mean',
#     #    'engine_L_engine_L_std', 'engine_L_engine_L_min',
#     #    'engine_L_engine_L_max', 'brand_engine_C_', 'engine_C_engine_C_median',
#     #    'engine_C_engine_C_mean', 'engine_C_engine_C_std',
#     #    'engine_C_engine_C_min', 'engine_C_engine_C_max'
#         ]]
# y = df["price"]


# # カテゴリカル変数をラベルエンコーディング
# label_encoders = {}
# for column in X.select_dtypes(include=['object']).columns:
#     le = LabelEncoder()
#     X[column] = le.fit_transform(X[column])
#     label_encoders[column] = le

# # 訓練セットとテストセットに分割
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.03, random_state=43)

# # 目的関数の定義
# def objective(trial):
#     param = {
#         'objective': 'regression',
#         'metric': 'rmse',
#         'num_leaves': trial.suggest_int('num_leaves',11, 90),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 300),
#         'min_child_samples': trial.suggest_int('min_child_samples', 20, 40),
#         'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
#         'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0)
#     }

#     model = lgb.LGBMRegressor(**param)
#     model.fit(X_train, y_train, eval_set=[(X_test, y_test)])
#     y_pred = model.predict(X_test)
#     mse = mean_squared_error(y_test, y_pred)
#     return mse

# # Optunaによるハイパーパラメータのチューニング
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=50)

# # 最良のハイパーパラメータを表示
# print("Best Hyperparameters:", study.best_params)

In [25]:
# 最良モデルの作成
best_params = study.best_params
best_model = lgb.LGBMRegressor(**best_params)

# モデルのトレーニング
best_model.fit(X_train, y_train, eval_set=[(X_test, y_test)])

# テストデータでの予測
y_pred = best_model.predict(X_test)

# 予測結果の表示
print("Predictions:", y_pred)

# 平均二乗誤差の計算
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse:.2f}')

NameError: name 'study' is not defined

In [144]:
test_df = pd.read_csv("./test.csv")
test_df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,54273,Mercedes-Benz,E-Class E 350,2014,73000,Gasoline,302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes
1,54274,Lexus,RX 350 Base,2015,128032,Gasoline,275.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Silver,Black,None reported,Yes
2,54275,Mercedes-Benz,C-Class C 300,2015,51983,Gasoline,241.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Blue,White,None reported,Yes
3,54276,Land,Rover Range Rover 5.0L Supercharged Autobiogra...,2018,29500,Gasoline,518.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,White,At least 1 accident or damage reported,Yes
4,54277,BMW,X6 xDrive40i,2020,90000,Gasoline,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes


In [145]:
brand_data = pd.DataFrame(df[["price", "brand"]].groupby("brand").agg({'price': ['count', 'median', 'mean', 'std', 'min', 'max']}).reset_index(drop=False))
brand_data.columns = ['_b_'.join(col).strip() for col in brand_data.columns.values]
brand_data = brand_data.fillna(0)
test_df = pd.merge(test_df, brand_data, how="left", left_on="brand", right_on="brand_b_")

In [146]:
for c in ["transmission_a", 'engine_hp', 'engine_L', 'engine_C']:
    brand_data = pd.DataFrame(df[[c , "brand"]].groupby("brand").agg({c : ['median', 'mean', 'std', "min", 'max']}).reset_index(drop=False))
    k = "_" + c + "_"
    brand_k = "brand" + k
    brand_data.columns = [k.join(col).strip() for col in brand_data.columns.values]
    brand_data = brand_data.fillna(0)
    test_df = pd.merge(test_df, brand_data, how="left", left_on="brand", right_on=brand_k)

In [147]:
test_df["accident_flg"] = test_df["accident"].apply(lambda x: 0 if x == "None reported" else 1)
test_df["transmission_a"] = test_df["transmission"].apply(make_transmission)
test_df["transmission_b"] = test_df["transmission"].apply(make_auto_manual)
test_df["sim_ext_col"] = test_df["ext_col"].apply(make_simple_color)
test_df["sim_int_col"] = test_df["int_col"].apply(make_simple_color)
test_df["year_diff"] = 2024 - test_df["model_year"]
test_df["year_diff_milage"] = test_df["year_diff"] * df["milage"]
test_df["engine_hp"] = test_df.apply(lambda row: make_engine_hp(row["engine"], "HP"), axis=1)
test_df["engine_L"] = test_df.apply(lambda row: make_engine_L(row["engine"], "L"), axis=1)
test_df["engine_C"] = test_df.apply(lambda row: make_engine_C(row["engine"], " Cylinder"), axis=1)
test_df["model_year_cls"] = test_df["model_year"].apply(make_my_class)
test_df["milage_cls"] = test_df["milage"].apply(make_milage_class)

In [148]:
test_df.columns

Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'brand_b_', 'price_b_count', 'price_b_median', 'price_b_mean',
       'price_b_std', 'price_b_min', 'price_b_max', 'brand_transmission_a_',
       'transmission_a_transmission_a_median',
       'transmission_a_transmission_a_mean',
       'transmission_a_transmission_a_std',
       'transmission_a_transmission_a_min',
       'transmission_a_transmission_a_max', 'brand_engine_hp_',
       'engine_hp_engine_hp_median', 'engine_hp_engine_hp_mean',
       'engine_hp_engine_hp_std', 'engine_hp_engine_hp_min',
       'engine_hp_engine_hp_max', 'brand_engine_L_',
       'engine_L_engine_L_median', 'engine_L_engine_L_mean',
       'engine_L_engine_L_std', 'engine_L_engine_L_min',
       'engine_L_engine_L_max', 'brand_engine_C_', 'engine_C_engine_C_median',
       'engine_C_engine_C_mean', 'engine_C_engine_C_std',
       'engine_C_engine_C

In [149]:
test_X = test_df[['brand', 'model_year', 'model', 'milage', 'fuel_type',
        'engine_hp', 'engine_L', 'engine_C',
        'transmission', 'transmission_a', 'transmission_b',
        'ext_col', 'int_col', 'sim_ext_col', 'sim_int_col',
        'accident',
        'year_diff',
        # 'milage_cls', 'model_year_cls',
        'price_b_count',
        'price_b_median', 'price_b_mean', 'price_b_std', 'price_b_min', 'price_b_max',
        # 'price_m_median', 'price_m_mean', 'price_m_std',
        # 'year_diff_milage',
    #    'brand_transmission_a_', 'transmission_a_transmission_a_median',
    #    'transmission_a_transmission_a_mean',
    #    'transmission_a_transmission_a_std',
    #    'transmission_a_transmission_a_min',
    #    'transmission_a_transmission_a_max', 'brand_engine_hp_',
    #    'engine_hp_engine_hp_median', 'engine_hp_engine_hp_mean',
    #    'engine_hp_engine_hp_std', 'engine_hp_engine_hp_min',
    #    'engine_hp_engine_hp_max', 'brand_engine_L_',
    #    'engine_L_engine_L_median', 'engine_L_engine_L_mean',
    #    'engine_L_engine_L_std', 'engine_L_engine_L_min',
    #    'engine_L_engine_L_max', 'brand_engine_C_', 'engine_C_engine_C_median',
    #    'engine_C_engine_C_mean', 'engine_C_engine_C_std',
    #    'engine_C_engine_C_min', 'engine_C_engine_C_max'
        ]]
# カテゴリカル変数をラベルエンコーディング
# label_encoders = {}
# for column in test_X.select_dtypes(include=['object']).columns:
#     le = LabelEncoder()
#     test_X[column] = le.fit_transform(test_X[column])
#     label_encoders[column] = le

In [150]:
# テストセットを使った予測
test_df["price"] = model.predict(test_X)
# test_df["price"] = test_df["price"].apply(lambda x : math.exp(x))

In [151]:
result_df = test_df[['id', 'price']]
result_df.to_csv("submission.csv", index=False)

In [152]:
result_df

,id,price
0,54273,23400.70
1,54274,19961.83
2,54275,28036.95
3,54276,71075.97
4,54277,39433.65
...,...,...
36178,90451,80398.94
36179,90452,17499.03
36180,90453,13412.79
36181,90454,56290.18


In [135]:
test_df.clean_title.value_counts()

clean_title
Yes    36183
Name: count, dtype: int64

In [136]:
best_params

{'num_leaves': 51,
 'learning_rate': 0.014198298257968938,
 'n_estimators': 169,
 'min_child_samples': 37,
 'subsample': 0.6228226625513161,
 'colsample_bytree': 0.6501340287840285}